### 날짜별, 기업별 기업 규모 열

In [1]:
import pandas as pd

In [5]:
import pandas as pd

# 파일 읽기
df = pd.read_csv('C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/스캔들/ALL논란_Final.csv', parse_dates=['사건 날짜'])

# 소속사(기업명)별 날짜 범위 확인
result = df.groupby('소속사')['사건 날짜'].agg(['min', 'max']).reset_index()

# 결과 출력
result

,소속사,min,max
0,HYBE,2021-02-22,2024-09-07
1,JYP,2016-02-21,2025-03-02
2,RBW,2023-05-12,2023-10-31
3,SM,2014-12-09,2025-04-16
4,SM C&C,2016-05-23,2023-08-15
5,YG,2014-08-22,2025-01-16
6,아티스트컴퍼니,2023-11-27,2024-11-24
7,큐브,2014-07-28,2023-12-15
8,키이스트,2014-05-30,2025-03-28


시총 데이터 하나로 묶기

In [18]:
import pandas as pd
import glob
import os

# 1. CSV 파일들이 있는 폴더 경로
folder_path = 'C:/Users/chica/Downloads'  # 실제 경로로 변경하세요
file_list = glob.glob(os.path.join(folder_path, '*.csv'))

# 2. 회사명별 DataFrame 저장용 딕셔너리
company_data = {}

# 3. 파일 하나씩 처리
for file in file_list:
    filename = os.path.basename(file)
    company_name = filename.split('_')[0]  # 언더바 이전 텍스트 추출

    # CSV 읽기 (인코딩 처리)
    try:
        df = pd.read_csv(file, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(file, encoding='cp949')

    # 딕셔너리에 누적
    if company_name in company_data:
        company_data[company_name].append(df)
    else:
        company_data[company_name] = [df]

# 4. 회사별로 병합해서 저장
for name, df_list in company_data.items():
    merged_df = pd.concat(df_list, ignore_index=True)
    merged_df.to_csv(f'{name}_시가총액.csv', index=False, encoding='utf-8-sig')
    print(f'{name}_시가총액.csv 저장 완료')


HYBE_시가총액.csv 저장 완료
JYP_시가총액.csv 저장 완료
RBW_시가총액.csv 저장 완료
SM C&C_시가총액.csv 저장 완료
SM_시가총액.csv 저장 완료
YG_시가총액.csv 저장 완료
아티스트컴퍼니_시가총액.csv 저장 완료
큐브_시가총액.csv 저장 완료
키이스트_시가총액.csv 저장 완료


In [22]:
hybe = pd.read_csv("HYBE_시가총액.csv", parse_dates=['일자'])
hybe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876 entries, 0 to 875
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      876 non-null    datetime64[ns]
 1   종가      876 non-null    int64         
 2   대비      876 non-null    int64         
 3   등락률     876 non-null    float64       
 4   시가      876 non-null    int64         
 5   고가      876 non-null    int64         
 6   저가      876 non-null    int64         
 7   거래량     876 non-null    int64         
 8   거래대금    876 non-null    int64         
 9   시가총액    876 non-null    int64         
 10  상장주식수   876 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(9)
memory usage: 75.4 KB


In [23]:
import pandas as pd
import glob
import os

# 현재 디렉토리에서 *_시가총액.csv 파일 모두 찾기
file_list = glob.glob('*_시가총액.csv')

for file in file_list:
    try:
        # CSV 파일 읽기 (인코딩 자동 처리)
        try:
            df = pd.read_csv(file, encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(file, encoding='cp949')

        # 중복 행 제거
        df = df.drop_duplicates()

        # '일자' 열이 존재할 경우에만 정렬
        if '일자' in df.columns:
            # 일자 열이 문자열이면 datetime으로 변환
            df['일자'] = pd.to_datetime(df['일자'], errors='coerce')
            df = df.sort_values('일자')

        # 다시 같은 파일 이름으로 저장
        df.to_csv(file, index=False, encoding='utf-8-sig')
        print(f'{file} 처리 완료')

    except Exception as e:
        print(f'{file} 처리 중 오류 발생: {e}')

HYBE_시가총액.csv 처리 완료
JYP_시가총액.csv 처리 완료
RBW_시가총액.csv 처리 완료
SM C&C_시가총액.csv 처리 완료
SM_시가총액.csv 처리 완료
YG_시가총액.csv 처리 완료
아티스트컴퍼니_시가총액.csv 처리 완료
큐브_시가총액.csv 처리 완료
키이스트_시가총액.csv 처리 완료


In [33]:
def map_closet_date(scandal_date, stock_date):
    return stock_date[stock_date>=scandal_date].min()


In [ ]:
scandal = pd.read_csv("C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/스캔들/ALL논란_Final.csv", encoding='utf-8', parse_dates=['사건 날짜'])

In [39]:
import pandas as pd
import numpy as np
import os

# 1. 사건 데이터 불러오기
scandal = pd.read_csv("C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/스캔들/ALL논란_Final.csv", encoding='utf-8', parse_dates=['사건 날짜'])

# 2. 날짜 매핑 함수: 사건 이후 가장 가까운 날짜
def map_closest_date(scandal_date, stock_dates):
    return stock_dates[stock_dates >= scandal_date].min()

# 3. 결과 저장 리스트
mapped_dates = []
mapped_caps = []

# 4. 절대 경로 설정
stock_data_dir = r"C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/주가"
marketcap_data_dir = r"C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/코드/전처리"

# 5. 각 사건 행에 대해 처리
for idx, row in scandal.iterrows():
    firm = row['소속사']
    scandal_date = row['사건 날짜']

    # 주가 데이터 및 시가총액 데이터 파일 경로
    stock_file = os.path.join(stock_data_dir, f"{firm}.csv")
    cap_file = os.path.join(marketcap_data_dir, f"{firm}_시가총액.csv")

    # 기본값 설정
    mapped_date = pd.NaT
    mapped_cap = np.nan

    if not os.path.exists(stock_file):
        print(f"❗ 주가 파일 없음: {stock_file}")
        mapped_dates.append(mapped_date)
        mapped_caps.append(mapped_cap)
        continue

    if not os.path.exists(cap_file):
        print(f"❗ 시가총액 파일 없음: {cap_file}")
        mapped_dates.append(mapped_date)
        mapped_caps.append(mapped_cap)
        continue

    try:
        # 1) 주가 데이터 불러오기
        stock = pd.read_csv(stock_file, parse_dates=['날짜'])
        stock = stock.sort_values('날짜')

        # 2) 날짜 매핑
        mapped_date = map_closest_date(scandal_date, stock['날짜'])
        mapped_dates.append(mapped_date)

        if pd.isna(mapped_date):
            mapped_caps.append(np.nan)
            continue

        # 3) 시가총액 데이터 불러오기
        cap_data = pd.read_csv(cap_file, parse_dates=['일자'])

        # 4) 매핑된 날짜로 시가총액 찾기
        row_match = cap_data[cap_data['일자'] == mapped_date]

        if not row_match.empty:
            mapped_cap = row_match.iloc[0]['시가총액']
        else:
            mapped_cap = np.nan

        mapped_caps.append(mapped_cap)

    except Exception as e:
        print(f"❌ {firm} 처리 중 오류: {e}")
        mapped_dates.append(mapped_date)
        mapped_caps.append(mapped_cap)

# 6. 결과 열 추가
scandal['매핑된 주가 날짜'] = mapped_dates
scandal['시가총액'] = mapped_caps

# 7. 저장
scandal.to_csv("ALL논란_Final_시가총액매핑완료.csv", index=False, encoding='utf-8-sig')
print("✅ 저장 완료: ALL논란_Final_시가총액매핑완료.csv")


✅ 저장 완료: ALL논란_Final_시가총액매핑완료.csv


In [41]:
sum(scandal['시가총액'].isna())

4

결측치 채운 후

In [3]:
import pandas as pd
df = pd.read_csv("ALL논란_Final_시가총액매핑완료.csv")
sum(df['시가총액'].isna())

0

파일에 시가총액 열 붙이기

In [2]:
import pandas as pd

# 파일 경로
all_path = "C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/스캔들/ALL논란_Final_시가총액.csv"
test_path = "C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/이벤트 스터디 결과/사건별_정당성_검정결과_tmp (1).csv"

# CSV 파일 불러오기
df_all = pd.read_csv(all_path)
df_test = pd.read_csv(test_path)

# 열 이름 통일
df_test = df_test.rename(columns={
    '사건날짜': '사건 날짜',
    '사건내용': '사건 내용'
})

# 시가총액 값을 담을 열 추가
df_test['시가총액'] = None

# 각 행별로 매칭해서 시가총액 가져오기
for i in range(len(df_test)):
    firm = df_test.loc[i, '소속사']
    celeb = df_test.loc[i, '연예인 이름']
    date = df_test.loc[i, '사건 날짜']

    match = df_all[
        (df_all['소속사'] == firm) &
        (df_all['연예인 이름'] == celeb) &
        (df_all['사건 날짜'] == date)
    ]

    if not match.empty:
        df_test.at[i, '시가총액'] = match.iloc[0]['시가총액']

# 저장
df_test.to_csv("C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/이벤트 스터디 결과/사건별_정당성_검정결과_tmp.csv", 
               encoding='utf-8-sig', index=False)

In [3]:
sum(df_test['시가총액'].isna())

0